In [1]:
import numpy as np 
import pandas as pd 
from ipynb.fs.full.funciones_FF import feature_RatioDayMoments, feature_Basicos, feature_Basicos_Ult_7d
from ipynb.fs.full.funciones_CF import qty_adoptions

# Seteo del mes snapshot

In [2]:
mes_snapshot = 9  #9 para train 12 para scoreo

# Lectura de los csv

In [3]:
df_pageviews = pd.read_csv("./data/pageviews.csv",
                   parse_dates=["FEC_EVENT"])
df_devicedata = pd.read_csv("./data/device_data.csv",
                   parse_dates=["FEC_EVENT"])
df_conversiones = pd.read_csv("./data/conversiones.csv")

In [4]:
#Se eliminan los registros que no deben ser considerados.
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month <= mes_snapshot]
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month > mes_snapshot - 9]

df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month <= mes_snapshot]
df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month > mes_snapshot - 9]

# Creación del universo

In [5]:
def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

In [6]:
universo = universo_train(df_pageviews)
universo.head(10)

,user_id,snapshot_mes
0,0,9
1,1,9
2,2,9
3,3,9
4,4,9
5,5,9
6,6,9
7,7,9
8,8,9
9,9,9


# Features

In [7]:
# def feature_X(df):

In [8]:
# def feature_X1(df):

# Armar dataset

In [9]:
# FF methods
universo = feature_RatioDayMoments(universo,df_pageviews)
universo = feature_Basicos(universo,df_pageviews) 
universo = feature_Basicos_Ult_7d(universo,df_pageviews)

# CF methods
universo = qty_adoptions(universo, df_conversiones)

# check df
universo.head(10)              

,user_id,snapshot_mes,FEC_EVENT_MAX,CANT_TOTAL_EVER,CANT_TOTAL_ULT_7D,FEC_EVENT_MENOS_7D,RATIO_VISITA_MADRUGADA,RATIO_VISITA_MANANA,RATIO_VISITA_TARDE,RATIO_VISITA_NOCHE,...,CONTENT_CATEGORY_BOTTOM_Ult_7d_63,SITE_ID_Ult_7d_1,SITE_ID_Ult_7d_2,SITE_ID_Ult_7d_3,SITE_ID_Ult_7d_4,ON_SITE_SEARCH_TERM_Ult_7d_1,qty_1,qty_3,qty_6,qty_9
0,0,9,2018-09-30 20:13:42,4912,90,2018-09-23 20:13:42,0.0,0.466667,0.222222,0.311111,...,0.0,0.000000,0.244444,0.755556,0.0,1.0,0.0,0.0,0.0,0.0
1,1,9,2018-09-30 20:56:50,1634,26,2018-09-23 20:56:50,0.0,0.961538,0.000000,0.038462,...,0.0,0.115385,0.230769,0.653846,0.0,1.0,0.0,0.0,0.0,0.0
2,2,9,2018-09-28 00:04:58,1717,15,2018-09-21 00:04:58,0.2,0.466667,0.333333,0.000000,...,0.0,0.200000,0.266667,0.533333,0.0,1.0,0.0,0.0,0.0,0.0
3,3,9,2018-09-28 16:03:37,1554,61,2018-09-21 16:03:37,0.0,0.049180,0.950820,0.000000,...,0.0,0.016393,0.229508,0.754098,0.0,1.0,0.0,0.0,0.0,0.0
4,4,9,2018-09-27 22:15:33,2478,50,2018-09-20 22:15:33,0.0,0.240000,0.300000,0.460000,...,0.0,0.000000,0.420000,0.580000,0.0,1.0,0.0,0.0,0.0,0.0
5,5,9,2018-09-30 13:13:39,4250,72,2018-09-23 13:13:39,0.0,0.361111,0.486111,0.152778,...,0.0,0.041667,0.111111,0.847222,0.0,1.0,0.0,0.0,0.0,0.0
6,6,9,2018-09-28 15:36:45,1108,48,2018-09-21 15:36:45,0.0,0.395833,0.604167,0.000000,...,0.0,0.000000,0.145833,0.854167,0.0,1.0,0.0,0.0,0.0,0.0
7,7,9,2018-09-27 09:19:27,1793,11,2018-09-20 09:19:27,0.0,0.272727,0.727273,0.000000,...,0.0,0.000000,0.363636,0.636364,0.0,1.0,0.0,0.0,0.0,0.0
8,8,9,2018-09-26 12:41:21,1549,137,2018-09-19 12:41:21,0.0,0.000000,0.744526,0.255474,...,0.0,0.080292,0.131387,0.788321,0.0,1.0,0.0,0.0,0.0,0.0
9,9,9,2018-09-30 22:01:01,1591,40,2018-09-23 22:01:01,0.0,0.000000,0.325000,0.675000,...,0.0,0.050000,0.125000,0.825000,0.0,1.0,0.0,0.0,0.0,0.0


# Target

In [10]:
def target(df):
    df = pd.merge(df, df_conversiones, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df.mes.fillna(value= 99, inplace=True)
    df = df.astype({'mes': 'int32'}) 
    
    df['target'] = 0
    df.loc[df.mes == df.snapshot_mes +1, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +2, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +3, 'target'] = 1

    return (df.groupby(['user_id', 'snapshot_mes'])['target'].max()).to_frame().reset_index()

In [11]:
target_universo = target(universo)
universo = pd.merge(universo, target_universo, how='left', left_on=['user_id', 'snapshot_mes'], right_on=['user_id','snapshot_mes'])

In [12]:
universo.to_csv(r'dataset.csv', index = None, header=True) 

In [13]:
universo[universo.qty_1 == 1]

,user_id,snapshot_mes,FEC_EVENT_MAX,CANT_TOTAL_EVER,CANT_TOTAL_ULT_7D,FEC_EVENT_MENOS_7D,RATIO_VISITA_MADRUGADA,RATIO_VISITA_MANANA,RATIO_VISITA_TARDE,RATIO_VISITA_NOCHE,...,SITE_ID_Ult_7d_1,SITE_ID_Ult_7d_2,SITE_ID_Ult_7d_3,SITE_ID_Ult_7d_4,ON_SITE_SEARCH_TERM_Ult_7d_1,qty_1,qty_3,qty_6,qty_9,target
378,378,9,2018-09-28 22:53:28,751,138,2018-09-21 22:53:28,0.000000,0.166667,0.760870,0.072464,...,0.000000,0.152174,0.847826,0.000000,1.0,1.0,1.0,1.0,1.0,0
382,382,9,2018-09-28 10:44:23,4471,43,2018-09-21 10:44:23,0.000000,0.790698,0.209302,0.000000,...,0.023256,0.186047,0.790698,0.000000,1.0,1.0,1.0,1.0,1.0,0
397,397,9,2018-09-24 13:48:16,1056,31,2018-09-17 13:48:16,0.000000,0.032258,0.967742,0.000000,...,0.322581,0.064516,0.612903,0.000000,1.0,1.0,1.0,1.0,1.0,1
533,533,9,2018-09-28 18:32:29,2707,44,2018-09-21 18:32:29,0.022727,0.000000,0.500000,0.477273,...,0.136364,0.250000,0.613636,0.000000,1.0,1.0,1.0,1.0,1.0,0
548,548,9,2018-09-28 14:13:01,1532,48,2018-09-21 14:13:01,0.000000,0.000000,0.812500,0.187500,...,0.062500,0.291667,0.645833,0.000000,1.0,1.0,2.0,4.0,4.0,0
562,562,9,2018-09-30 19:01:46,1899,259,2018-09-23 19:01:46,0.000000,0.359073,0.401544,0.239382,...,0.069498,0.181467,0.749035,0.000000,1.0,1.0,2.0,2.0,2.0,0
565,565,9,2018-09-27 13:07:19,2282,52,2018-09-20 13:07:19,0.000000,0.692308,0.307692,0.000000,...,0.000000,0.307692,0.692308,0.000000,1.0,1.0,1.0,1.0,2.0,0
610,610,9,2018-09-28 13:19:49,3659,122,2018-09-21 13:19:49,0.000000,0.278689,0.696721,0.024590,...,0.040984,0.204918,0.754098,0.000000,1.0,1.0,1.0,3.0,3.0,0
637,637,9,2018-09-30 22:32:22,1394,148,2018-09-23 22:32:22,0.000000,0.466216,0.000000,0.533784,...,0.000000,0.175676,0.824324,0.000000,1.0,1.0,1.0,2.0,2.0,1
676,676,9,2018-09-28 18:15:51,2605,65,2018-09-21 18:15:51,0.000000,0.400000,0.538462,0.061538,...,0.123077,0.307692,0.569231,0.000000,1.0,1.0,1.0,1.0,1.0,0
